In [2]:
# -*- coding: utf-8 -*-
#
# Author: Milan Ondrasovic <milan.ondrasovic@gmail.com>
#
# MIT License
#
# Copyright (c) 2021 Milan Ondrašovič
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import numpy as np
import cv2 as cv

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
x_top = 200
y_top = 150
side = 200

dst_pts = (
    (x_top, y_top),
    (x_top + side, y_top),
    (x_top + side, y_top + side),
    (x_top, y_top + side),
)

dst_pts = np.asarray(dst_pts, dtype=np.int32)

def to_homogeneous(pts):
    return np.squeeze(cv.convertPointsToHomogeneous(pts)).T

def from_homogeneous(pts):
    return np.squeeze(cv.convertPointsFromHomogeneous(pts.T))

def affine_transform(pts, theta, t_x, t_y, s_x, s_y):
    c_x, c_y = pts.mean(axis=0)[:2]

    center = np.float32((
        (1, 0, -c_x),
        (0, 1, -c_y),
        (0, 0, 1)
    ))
    uncenter = np.float32((
        (1, 0, c_x),
        (0, 1, c_y),
        (0, 0, 1)
    ))

    rot = np.float32((
        (np.cos(theta), -np.sin(theta), 0),
        (np.sin(theta), np.cos(theta),  0),
        (0,             0,              1)
    ))
    transl = np.float32((
        (1, 0, t_x),
        (0, 1, t_y),
        (0, 0, 1)
    ))
    scale = np.float32((
        (s_x, 0,   0),
        (0,   s_y, 0),
        (0,   0,   1)
    ))

    affine = transl @ uncenter @ scale @ rot @ center
    return from_homogeneous(affine @ to_homogeneous(pts))

src_pts = dst_pts.copy()
src_pts = affine_transform(src_pts, np.radians(45), 200, 200, 2, 2)
src_pts = src_pts.round().astype(np.int)
print(src_pts)

def draw_points(
        img, pts, line_color=(255, 255, 255), thickness=2,
        point_color=(255, 255, 255)):
    cv.polylines(
        img, [np.expand_dims(pts, axis=1)], isClosed=True,
        color=line_color, thickness=thickness, lineType=cv.LINE_AA)
    for pt in np.squeeze(pts):
        cv.circle(img, tuple(pt), radius=thickness * 3, color=point_color,
                  thickness=-1, lineType=cv.LINE_AA)

img_width = 800
img_height = 600

# src_pts = src_pts[np.random.permutation(len(src_pts))]
affine_mtx, _ = cv.estimateAffine2D(src_pts, dst_pts)
affine_mtx = np.concatenate((affine_mtx, [(0, 0, 1)]))

src_pts_t = from_homogeneous(affine_mtx @ to_homogeneous(src_pts))
src_pts_t = src_pts_t.round().astype(np.int)

img = np.zeros((img_height, img_width, 3), dtype=np.uint8)
draw_points(img, dst_pts, line_color=(0, 255, 0), thickness=10)
draw_points(img, src_pts, line_color=(255, 0, 0))
draw_points(img, src_pts_t, line_color=(0, 0, 255))
cv.imshow("Preview", img)
cv.waitKey(0)
cv.destroyAllWindows()

[[500 167]
 [783 450]
 [500 733]
 [217 450]]


In [40]:
p = np.random.permutation(len(src_pts_t))
src_pts_t[p]

array([[323, 335],
       [123, 335],
       [400, 150],
       [200, 150]])

In [42]:
def print_score(pts_1, pts_2, label_1, label_2):
    score = np.mean(np.linalg.norm(pts_1 - pts_2, axis=1))
    print(f"SCORE({label_1}, {label_2}) = {score:.6f}")

print_score(src_pts, dst_pts, "SRC", "DST")
print_score(src_pts_t, dst_pts, "SRC_T", "DST")

SCORE(SRC, DST) = 297.098891
SCORE(SRC_T, DST) = 63.245553


In [24]:
from datagen.transform import Scale
from datagen.shape import RectangleGenerator

scale = Scale(1.5, 2.0, use_centroid=True)
rect_gen = RectangleGenerator(100, 100)
center = np.asarray((200, 200))
rect = rect_gen.generate_points(center)
print(rect)
print(scale.transform_points_group(rect))
pts = scale.transform_points_group(rect).round().astype(np.int)
img = np.zeros((img_height, img_width, 3), dtype=np.uint8)
draw_points(img, rect.round().astype(np.int))
draw_points(img, pts)
cv.imshow("Preview", img)
cv.waitKey(0)
cv.destroyAllWindows()

[[150. 150.]
 [250. 150.]
 [250. 250.]
 [150. 250.]]
[[125. 100.]
 [275. 100.]
 [275. 300.]
 [125. 300.]]


In [52]:
def create_similarity_transform(theta, t_x, t_y, s):
    rot = np.float32((
        (np.cos(theta), -np.sin(theta), 0),
        (np.sin(theta), np.cos(theta),  0),
        (0,             0,              1)
    ))
    transl = np.float32((
        (1, 0, t_x),
        (0, 1, t_y),
        (0, 0, 1)
    ))
    scale = np.float32((
        (s, 0, 0),
        (0, s, 0),
        (0, 0, 1)
    ))

    return scale @ transl @ rot

mtx = create_similarity_transform(np.radians(45), 10, 10, 1)
mtx

array([[ 0.70710677, -0.70710677, 10.        ],
       [ 0.70710677,  0.70710677, 10.        ],
       [ 0.        ,  0.        ,  1.        ]], dtype=float32)

In [37]:
import time
import math

def sum_sqrt(n_iters=1_000_000):
    return sum(map(math.sqrt, range(n_iters)))

start_time = time.process_time_ns()
st = time.time()
print(f"sum of square roots: {sum_sqrt()}")
time.sleep(1)
elapsed_time_ns = time.process_time_ns() - start_time
elapsed_time = time.time() - st

sum of square roots: 666666166.4588418


In [38]:
elapsed_time_ns, elapsed_time

(93750000, 1.0923335552215576)